In [7]:
import re
import numpy as np 
import pandas as pd
import random
import math
from google.colab import files
import io
from matplotlib import pyplot as plt

In [8]:
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['H2_train.csv']))

Saving H2_train.csv to H2_train.csv


In [9]:
df.head(10)

,text,label,id
0,"ऑटोरिक्शा चालक जावेद खान कहते हैं, ""कोविड-19 क...",NOT,0
1,@Raghuveerdutt62 @Arunesh24797107 रघुवीर जी यह...,NOT,1
2,@Drmahera552 @common000786 मोदी तुम इस्तीफा दो...,NOT,2
3,@sab_hateDMS Does she wear mesh hooker hose an...,HOF,3
4,What a bunch of absolute fucking idiots in #in...,HOF,4
5,I donâ€™t fuck with anyone and yet I still hea...,HOF,5
6,We never had our priorities right. Never. #Res...,HOF,6
7,"किसी ने नहीं सोचा था ये दिन भी देखना पड़ेगा, म...",NOT,7
8,@China_Amb_India @narendramodi I am shocked th...,NOT,8
9,अच्छा भई @JitinPrasada बताओ यार अब @myogiadity...,HOF,9


In [10]:
df = df.drop( columns=['id'])

# **Translator**

In [12]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.2 MB/s 
     |████████████████████████████████| 1.5 MB 12.9 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=74cb1158fa36f189d7e30006b48dcecac1364f5488ec4c1958e46cdebcc09f1b
  Stored in directory: /root/.cache/pip/wheels/dd/59/af/8d6c96a719763990f1c548e36b17d9efdfb767f42f7ff39f53
Successfully built googletrans


In [13]:
from googletrans import Translator
translator = Translator()

In [14]:
def english_translate(tweet):
  out = translator.translate(tweet)
  return out.text

In [16]:
for i in range(len(df)):
  df['text'][i] = english_translate(df['text'][i])

In [17]:
df.head()

,text,label
0,"Autorickshaw driver Javed Khan says, ""A seriou...",NOT
1,@Raghuveerdutt62 @Arunesh24797107 Raghuveer ji...,NOT
2,@Drmahera552 @common000786 Modi you resign we ...,NOT
3,@sab_hateDMS Does she wear mesh hooker hose an...,HOF
4,What a bunch of absolute fucking idiots in #in...,HOF


In [20]:
df.isnull().values.any()

False

In [21]:
df.to_csv('train_trans.csv')

# **pre processing**

In [14]:
!pip install tweet-preprocessor
import preprocessor as p

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

In [71]:
df['text'] = df.apply(preprocess_tweet, axis=1)

In [72]:
df.head(10)

,text,label
0,"Autorickshaw driver Javed Khan says, ""A seriou...",NOT
1,"Raghuveer ji, this is the truth that monkeys w...",NOT
2,Modi you resign we are with you,NOT
3,Does she wear mesh hooker hose and have a Phd ...,HOF
4,What a bunch of absolute fucking idiots in . B...,HOF
5,I dont fuck with anyone and yet I still hear a...,HOF
6,We never had our priorities right. Never.,HOF
7,No one had thought that this day will also hav...,NOT
8,I am shocked they didn't mention a line (Karam...,NOT
9,"Well brother tell me, now will you attack on t...",HOF


In [47]:
df['text'][95]

"You bastards are not fit to take Yogi's name."

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
TV = TfidfVectorizer(min_df=3)   
X = TV.fit_transform(df['text']).toarray()
y = df['label']
print(np.shape(X))
print(np.shape(y))

(7593, 4328)
(7593,)


In [74]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)    # standardise the data

In [75]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size = 0.75, test_size = 0.25)

In [88]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
dt = RandomForestClassifier( )
dt.fit(x_train, y_train)

RandomForestClassifier()

In [83]:
from sklearn.metrics import classification_report

In [89]:
y_pred = dt.predict(x_train)

In [90]:
print(classification_report(y_train , y_pred))

              precision    recall  f1-score   support

         HOF       0.99      0.98      0.99      2660
         NOT       0.98      0.99      0.99      3034

    accuracy                           0.99      5694
   macro avg       0.99      0.99      0.99      5694
weighted avg       0.99      0.99      0.99      5694



In [91]:
y_pred = dt.predict(x_val)

In [92]:
print(classification_report(y_val , y_pred))

              precision    recall  f1-score   support

         HOF       0.75      0.68      0.71       869
         NOT       0.75      0.81      0.78      1030

    accuracy                           0.75      1899
   macro avg       0.75      0.75      0.75      1899
weighted avg       0.75      0.75      0.75      1899



In [93]:
from sklearn. ensemble import AdaBoostClassifier
adb = AdaBoostClassifier(DecisionTreeClassifier())
adb.fit(x_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier())

In [94]:
y_pred = dt.predict(x_train)
print(classification_report(y_train , y_pred))

              precision    recall  f1-score   support

         HOF       0.99      0.98      0.99      2660
         NOT       0.98      0.99      0.99      3034

    accuracy                           0.99      5694
   macro avg       0.99      0.99      0.99      5694
weighted avg       0.99      0.99      0.99      5694



In [95]:
y_pred = dt.predict(x_val)
print(classification_report(y_val , y_pred))

              precision    recall  f1-score   support

         HOF       0.75      0.68      0.71       869
         NOT       0.75      0.81      0.78      1030

    accuracy                           0.75      1899
   macro avg       0.75      0.75      0.75      1899
weighted avg       0.75      0.75      0.75      1899



In [52]:
y_pred[:15]

array(['NOT', 'NOT', 'NOT', 'HOF', 'HOF', 'HOF', 'HOF', 'NOT', 'NOT',
       'HOF', 'HOF', 'HOF', 'NOT', 'NOT', 'HOF'], dtype=object)

In [22]:
uploaded = files.upload()
df2 = pd.read_csv(io.BytesIO(uploaded['H2_test.csv']))

Saving H2_test.csv to H2_test.csv


In [23]:
for i in range(len(df2)):
  df2['text'][i] = english_translate(df2['text'][i])

<ipython-input-23-da51db0acb5d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['text'][i] = english_translate(df2['text'][i])


In [106]:
df2['text'] = df2.apply(preprocess_tweet, axis=1)

In [24]:
df2.head(10)

,text,id
0,@Jigeumun_ Well at least u knew he was shitty ...,0
1,"As a humble worker of BJP, I am registering my...",1
2,@AnandPa44510218 @Vitticollis @JBhimJBharat @P...,2
3,fucker https://t.co/oy2XTmKlbo,3
4,"This is the condition of winning, just think w...",4
5,Nooooooooo Noooooooooo!!!! Very few nationalis...,5
6,@ZeeNewsEnglish @sudhirchaudhary Please unders...,6
7,Why Modi as Prime Minister is big Disaster for...,7
8,"@AnjaliBirla4 Election rally, khumba mela etc ...",8
9,@FrankDTrombly1 @RJTrustedAdvice @PalmerReport...,9


In [25]:
df2.to_csv('test_trans.csv')

In [102]:
df2 = df2.drop(columns = ['id'])

In [103]:
df2.head(10)

,text
0,Well at least u knew he was shitty i couldn't ...
1,"As a humble worker of BJP, I am registering my..."
2,
3,fucker
4,"This is the condition of winning, just think w..."
5,Nooooooooo Noooooooooo!!!! Very few nationalis...
6,Please understand by now that power and money ...
7,Why Modi as Prime Minister is big Disaster for...
8,"Election rally, khumba mela etc made the situa..."
9,"Trump paid $180,000 for sex but you think he's..."


In [60]:
X_test = TV.fit_transform(df['text']).toarray()
print(np.shape(X_test))

In [62]:
X_test = StandardScaler().fit_transform(X_test)    # standardise the data

In [63]:
y_pred = dt.predict(X_test)

ValueError: ignored

In [ ]:
import csv
header = ['label', 'id']

with open('result.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)
    
    # write the rows
    for i in range(len(dataset_B)):
      writer.writerow([dataset_B[i], label_B[i]])

In [1]:
df.head(10)

NameError: ignored

In [97]:
df.to_csv('train.csv')

In [4]:
uploaded = files.upload()
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

Saving train.csv to train.csv


KeyError: ignored

In [6]:
uploaded = files.upload()
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

Saving test.csv to test.csv
